# Optimizer（优化器）
## 优化器的目的
   **优化器是为了一步一步走到代价函数的最小值那里**
![6_1](image/class6_1.png)
## 各种优化器
1. 标准梯度下降法
    - 标准梯度下降先计算所有样本汇总误差，然后根据总误差来更新权值
    `tf.train.GradientDescentOptimizer`
2. 随机梯度下降法
    - 随机梯度下降随机抽取一个样本来计算误差，然后更新权值
    - 随机梯度下降的公式
 ![6-2](image/class6_2.png)
3. 批量梯度下降法
    - 批量梯度下降是一种折中的方案，从总样本中选取一个批次（比如一共有10000个样本，随机选取100个样本作为个batch），然后计算这个batch的总误差，根据总误差来更新权值
 
 ![6-3](image/class6_3.png)
 ![6-4](image/class6_4.png)
 ![6-5](image/class6_5.png)
 ![6-6](image/class6_6.png)
 ![6-7](image/class6_7.png)


## 优化器的使用

In [2]:
# 优化器的使用案例
# 手写数字识别案例(线性分类器)
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
# 载入数据集。第一个参数是路径名，第二个是采用的编码
mnist = input_data.read_data_sets("/Users/yuejinxiong/MNIST_data",one_hot=True)

# 每个批次的大小(每次放入100张图片去训练)
batch_size = 100
# 计算一共有多少个批次
n_batch = mnist.train.num_examples // batch_size

# 定义两个placeholder
x = tf.placeholder(tf.float32, [None,784])
y = tf.placeholder(tf.float32, [None,10])
# keep_prob = tf.placeholder(tf.float32)

# 创建一个简单的神经网络
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))
prediction = tf.nn.softmax(tf.matmul(x, W)+b)


# # 第一层神经元
# # 用截断的正态分布来初始化,其标准差是0.1
# W1 = tf.Variable(tf.truncated_normal([784, 2000],stddev=0.1))
# # 偏置值初始化为0.1
# b1 = tf.Variable(tf.zeros([2000])+0.1)
# L1 = tf.nn.tanh(tf.matmul(x, W1)+b1)
# # 调用封装好的dropout函数,其中的keep_prob参数就是设置让 百分之几的神经元工作,如：0.5就是50%的神经元工作
# L1_drop = tf.nn.dropout(L1, keep_prob)

# # 第二层神经元
# W2 = tf.Variable(tf.truncated_normal([2000, 2000],stddev=0.1))
# b2 = tf.Variable(tf.zeros([2000])+0.1)
# L2 = tf.nn.tanh(tf.matmul(L1_drop, W2)+b2)
# L2_drop = tf.nn.dropout(L2, keep_prob)

# # 第三层神经元(用这么多层神经元,每层这么多神经元,就是为了模拟过拟合)
# W3 = tf.Variable(tf.truncated_normal([2000, 1000],stddev=0.1))
# b3 = tf.Variable(tf.zeros([1000])+0.1)
# L3 = tf.nn.tanh(tf.matmul(L2_drop, W3)+b3)
# L3_drop = tf.nn.dropout(L3, keep_prob)

# # 第四层神经元(用这么多层神经元,每层这么多神经元,就是为了模拟过拟合)
# W4 = tf.Variable(tf.truncated_normal([1000, 10],stddev=0.1))
# b4 = tf.Variable(tf.zeros([10])+0.1)
# prediction = tf.nn.softmax(tf.matmul(L3_drop, W4)+b4)


# 增加了隐藏层之后,参数变多了,更加难以收敛,如果想得到好的结果,必须要迭代很多次
# 隐藏层的神经元个数最好是2^n个

# 输入层与隐藏层之间的权重和偏置的初始化 隐藏层有5个神经元
# Weight_L1 = tf.Variable(tf.random_normal([784, 512]))
# biases_L1 = tf.Variable(tf.zeros([1, 512]))
# Wx_plus_b_L1 = tf.matmul(x, Weight_L1) + biases_L1
# L1 = tf.nn.tanh(Wx_plus_b_L1)

# 隐藏层和输出层之间权重和偏置的初始化
# Weight_L2 = tf.Variable(tf.random_normal([512, 10]))
# biases_L2 = tf.Variable(tf.zeros([1, 10]))
# Wx_plus_b_L2 = tf.matmul(L1, Weight_L2) + biases_L2
# prediction = tf.nn.softmax(Wx_plus_b_L2)

# 二次代价函数
# loss = tf.reduce_mean(tf.square(y-prediction))
# 现在改为对数似然函数来优化
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y, logits=prediction))
# 随机梯度下降法
# train_step = tf.train.GradientDescentOptimizer(0.2).minimize(loss)

# 此次不在使用普通的优化器而转用一些别的优化器(Adam) 10^-2
# 发现学习率虽然比上面的0.2更小，但是收敛速度却更快了
train_step = tf.train.AdamOptimizer(1e-2).minimize(loss)

# 初始化变量
init = tf.global_variables_initializer()

# 这里求预测的正确数、结果存在一个布尔型的列表中
# equal(num1, num2)比较两个参数的大小是否一样,返回值为True或False
# arg_max() 求y(一维张量)这个结果最大的值的索引位置(如果标签值和预测值最大的值在同一个位置，则说明该条正确) 1表示按行查找
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(prediction, 1))

# 求准确率, cast()将布尔类型的列表中的元素转成32位的浮点类型,然后再求一个平均值(true=1.0 false=0)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

with tf.Session() as sess:
    sess.run(init)
    # 所有数据迭代21次
    for epoch in range(21):
        # 一次迭代中迭代计算好的数量个批次
        for batch in range(n_batch):
            # 获取100个批次保存在里面,数据保存在batch_xs中，标签保存在batchys中
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            # 将keep_prob 分别设置为1.0（所有神经元均工作） 和0.5等等进行比较
            sess.run(train_step, feed_dict={x:batch_xs, y:batch_ys})
        # 训练一个周期后,可以查看其准确率的变化
        # 查看训练集的准确率
        # train_acc = sess.run(accuracy, feed_dict={x:mnist.train.images, y:mnist.train.labels, keep_prob:1.0})
        # 查看测试集的准确率
        test_acc = sess.run(accuracy, feed_dict={x:mnist.test.images, y:mnist.test.labels})
        print("第"+str(epoch+1)+"次迭代的测试集准确率: "+str(test_acc))

Extracting /Users/yuejinxiong/MNIST_data/train-images-idx3-ubyte.gz
Extracting /Users/yuejinxiong/MNIST_data/train-labels-idx1-ubyte.gz
Extracting /Users/yuejinxiong/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting /Users/yuejinxiong/MNIST_data/t10k-labels-idx1-ubyte.gz
第1次迭代的测试集准确率: 0.9189
第2次迭代的测试集准确率: 0.9242
第3次迭代的测试集准确率: 0.9253
第4次迭代的测试集准确率: 0.9294
第5次迭代的测试集准确率: 0.9243
第6次迭代的测试集准确率: 0.9312
第7次迭代的测试集准确率: 0.9314
第8次迭代的测试集准确率: 0.9306
第9次迭代的测试集准确率: 0.9314
第10次迭代的测试集准确率: 0.9301
第11次迭代的测试集准确率: 0.9289
第12次迭代的测试集准确率: 0.9317
第13次迭代的测试集准确率: 0.9324
第14次迭代的测试集准确率: 0.9326
第15次迭代的测试集准确率: 0.9323
第16次迭代的测试集准确率: 0.9313
第17次迭代的测试集准确率: 0.9316
第18次迭代的测试集准确率: 0.932
第19次迭代的测试集准确率: 0.9308
第20次迭代的测试集准确率: 0.9324
第21次迭代的测试集准确率: 0.9309


# 小结
## 目前的几种提高模型的准确率和训练速度的方法
1. 更改损失函数，原先使用二次代价函数，对于softmax或者sigmoid可以使用交叉熵来作为代价函数
2. 权值矩阵W一般最好初始化为截断的随机正态分布；偏置b一般可以初始化为0.1，而不是用0来初始化
3. 适当增加隐藏层可以更好的拟合函数，提高模型的准确率
4. 更改激活函数，找到适合模型的激活函数：sigmoid、tanh或者Relu等
5. 防止过拟合问题，在训练时可以采用dropout来随机使一些神经元失去活性
6. 优化器不再使用随机梯度下降而改用其他的，明显提升梯度下降的速度；优化器的学习率也可以适当调整

# 使用上述小结中的方法来优化上面的模型使其准确率达到98%以上

In [5]:
# 综合优化案例
# 手写数字识别案例(线性分类器)
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

# 载入数据集。第一个参数是路径名，第二个是采用的编码
mnist = input_data.read_data_sets("/Users/yuejinxiong/MNIST_data",one_hot=True)

# 每个批次的大小(每次放入100张图片去训练)
batch_size = 100
# 计算一共有多少个批次
n_batch = mnist.train.num_examples // batch_size

# 定义两个placeholder
x = tf.placeholder(tf.float32, [None,784])
y = tf.placeholder(tf.float32, [None,10])
# keep_prob = tf.placeholder(tf.float32)
lr = tf.Variable(0.001, dtype=tf.float32)

# 创建一个简单的神经网络
# W = tf.Variable(tf.zeros([784, 10]))
# b = tf.Variable(tf.zeros([10]))
# prediction = tf.nn.softmax(tf.matmul(x, W)+b)


# # 第一层神经元
# # 用截断的正态分布来初始化,其标准差是0.1
# W1 = tf.Variable(tf.truncated_normal([784, 2000],stddev=0.1))
# # 偏置值初始化为0.1
# b1 = tf.Variable(tf.zeros([2000])+0.1)
# L1 = tf.nn.tanh(tf.matmul(x, W1)+b1)
# # 调用封装好的dropout函数,其中的keep_prob参数就是设置让 百分之几的神经元工作,如：0.5就是50%的神经元工作
# L1_drop = tf.nn.dropout(L1, keep_prob)

# # 第二层神经元
# W2 = tf.Variable(tf.truncated_normal([2000, 2000],stddev=0.1))
# b2 = tf.Variable(tf.zeros([2000])+0.1)
# L2 = tf.nn.tanh(tf.matmul(L1_drop, W2)+b2)
# L2_drop = tf.nn.dropout(L2, keep_prob)

# # 第三层神经元(用这么多层神经元,每层这么多神经元,就是为了模拟过拟合)
# W3 = tf.Variable(tf.truncated_normal([2000, 1000],stddev=0.1))
# b3 = tf.Variable(tf.zeros([1000])+0.1)
# L3 = tf.nn.tanh(tf.matmul(L2_drop, W3)+b3)
# L3_drop = tf.nn.dropout(L3, keep_prob)

# # 第四层神经元(用这么多层神经元,每层这么多神经元,就是为了模拟过拟合)
# W4 = tf.Variable(tf.truncated_normal([1000, 10],stddev=0.1))
# b4 = tf.Variable(tf.zeros([10])+0.1)
# prediction = tf.nn.softmax(tf.matmul(L3_drop, W4)+b4)


# 增加了隐藏层之后,参数变多了,更加难以收敛,如果想得到好的结果,必须要迭代很多次
# 隐藏层的神经元个数最好是2^n个

# 优化1:增加一层隐藏层，隐藏层神经元1024个；优化2:初始化权重矩阵W和偏置b时分别采用截断的随机正态分布和0.1
# 输入层与隐藏层之间的权重和偏置的初始化 隐藏层有5个神经元
Weight_L1 = tf.Variable(tf.truncated_normal([784, 1024], stddev=0.1))
biases_L1 = tf.Variable(tf.zeros([1, 1024]) + 0.1)
Wx_plus_b_L1 = tf.matmul(x, Weight_L1) + biases_L1
L1 = tf.nn.tanh(Wx_plus_b_L1)

# 隐藏层和输出层之间权重和偏置的初始化
Weight_L2 = tf.Variable(tf.truncated_normal([1024, 10], stddev=0.1))
biases_L2 = tf.Variable(tf.zeros([1, 10]) + 0.1)
Wx_plus_b_L2 = tf.matmul(L1, Weight_L2) + biases_L2
prediction = tf.nn.softmax(Wx_plus_b_L2)

# 二次代价函数
# loss = tf.reduce_mean(tf.square(y-prediction))
# 优化3: 现在改为softmax的专用的对数似然函数来优化
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y, logits=prediction))

# 随机梯度下降法
# train_step = tf.train.GradientDescentOptimizer(0.2).minimize(loss)

# 优化4:此次不在使用普通的优化器而转用一些别的优化器(Adam) 10^-2
# 发现学习率虽然比上面的0.2更小，但是收敛速度却更快了 lr-->1e-3
train_step = tf.train.AdamOptimizer(1e-3).minimize(loss)

# 初始化变量
init = tf.global_variables_initializer()

# 这里求预测的正确数、结果存在一个布尔型的列表中
# equal(num1, num2)比较两个参数的大小是否一样,返回值为True或False
# arg_max() 求y(一维张量)这个结果最大的值的索引位置(如果标签值和预测值最大的值在同一个位置，则说明该条正确) 1表示按行查找
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(prediction, 1))

# 求准确率, cast()将布尔类型的列表中的元素转成32位的浮点类型,然后再求一个平均值(true=1.0 false=0)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

with tf.Session() as sess:
    sess.run(init)
    # 所有数据迭代21次
    for epoch in range(21):
        # 优化5:初始的学习率为0.001，每迭代一个周期，就将学习率调低
        # sess.run(tf.assign(lr, 0.001*(0.095 ** epoch)))
        # 一次迭代中迭代计算好的数量个批次
        for batch in range(n_batch):
            # 获取100个批次保存在里面,数据保存在batch_xs中，标签保存在batchys中
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            # 将keep_prob 分别设置为1.0（所有神经元均工作） 和0.5等等进行比较
            sess.run(train_step, feed_dict={x:batch_xs, y:batch_ys})
        # 训练一个周期后,可以查看其准确率的变化
        # 查看训练集的准确率
        # train_acc = sess.run(accuracy, feed_dict={x:mnist.train.images, y:mnist.train.labels, keep_prob:1.0})
        # 查看测试集的准确率
        test_acc = sess.run(accuracy, feed_dict={x:mnist.test.images, y:mnist.test.labels})
        print("第"+str(epoch+1)+"次迭代的测试集准确率: "+str(test_acc)+";学习率为：**")
        # +str(sess.run(lr))

Extracting /Users/yuejinxiong/MNIST_data/train-images-idx3-ubyte.gz
Extracting /Users/yuejinxiong/MNIST_data/train-labels-idx1-ubyte.gz
Extracting /Users/yuejinxiong/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting /Users/yuejinxiong/MNIST_data/t10k-labels-idx1-ubyte.gz
第1次迭代的测试集准确率: 0.9387;学习率为：<tf.Variable 'Variable_11:0' shape=() dtype=float32_ref>
第2次迭代的测试集准确率: 0.9473;学习率为：<tf.Variable 'Variable_11:0' shape=() dtype=float32_ref>
第3次迭代的测试集准确率: 0.9478;学习率为：<tf.Variable 'Variable_11:0' shape=() dtype=float32_ref>
第4次迭代的测试集准确率: 0.9479;学习率为：<tf.Variable 'Variable_11:0' shape=() dtype=float32_ref>
第5次迭代的测试集准确率: 0.9479;学习率为：<tf.Variable 'Variable_11:0' shape=() dtype=float32_ref>
第6次迭代的测试集准确率: 0.9479;学习率为：<tf.Variable 'Variable_11:0' shape=() dtype=float32_ref>
第7次迭代的测试集准确率: 0.9479;学习率为：<tf.Variable 'Variable_11:0' shape=() dtype=float32_ref>
第8次迭代的测试集准确率: 0.9479;学习率为：<tf.Variable 'Variable_11:0' shape=() dtype=float32_ref>
第9次迭代的测试集准确率: 0.9479;学习率为：<tf.Variable 'Variable_11:0' shape=() dt